In [2]:
%pip install transformers

In [3]:
import pandas as pd
import re
from transformers import pipeline
import sys, os
sys.path.append(os.path.abspath(".."))

In [14]:
from google.colab import files
uploaded = files.upload()

Saving pii_test_data.csv to pii_test_data.csv


In [15]:
df = pd.read_csv("pii_test_data.csv")
df.head()

,conversation_id,timestamp,user_input,assistant_output,contains_pii,pii_types
0,conv_001,2024-11-15 09:23:45,"Hi, I need help resetting my password. My emai...",I can help you reset your password. I've sent ...,True,"email,username"
1,conv_002,2024-11-15 10:14:22,What's the weather like today?,The weather today is partly cloudy with temper...,False,none
2,conv_003,2024-11-15 11:05:33,I'm trying to update my profile. My name is Mi...,Thank you Michael. I've updated your profile w...,True,"name_student,phone_num,street_address"
3,conv_004,2024-11-15 12:45:18,Can you explain how machine learning works?,Machine learning is a subset of AI where syste...,False,none
4,conv_005,2024-11-15 13:22:09,"Hello, I need to verify my account. My ID numb...",I've verified your account using ID number DL-...,True,"id_num,url_personal"


In [1]:
from google.colab import files
uploaded = files.upload()

Saving pii_test_data(2).csv to pii_test_data(2).csv


In [13]:
df = pd.read_csv("pii_test_data(2).csv")
df.head()


,user_input,assistant_output
0,"Hi, I need help resetting my password. My emai...",I can help you reset your password. I've sent ...
1,What's the weather like today?,The weather today is partly cloudy with temper...
2,I'm trying to update my profile. My name is Mi...,Thank you Michael. I've updated your profile w...
3,Can you explain how machine learning works?,Machine learning is a subset of AI where syste...
4,"Hello, I need to verify my account. My ID numb...",I've verified your account using ID number DL-...


In [7]:

ner_pipeline = pipeline(
    "ner",
    model="./sample_data/pii_model",
    tokenizer="./sample_data/pii_model",
    aggregation_strategy="simple"
)

test_sentence = "Hi, my name is Minh and my email is minh123@gmail.com and phone 0908-123-456"
results = ner_pipeline(test_sentence)

for entity in results:
    print(f"Entity: {entity['word']}, Label: {entity['entity_group']}, Score: {entity['score']:.2f}")


Device set to use cuda:0


Entity: minh, Label: NAME_STUDENT, Score: 0.56
Entity: minh123 @ gmail. com, Label: EMAIL, Score: 0.94
Entity: phone, Label: PHONE_NUM, Score: 0.29
Entity: 0908 - 123 - 456, Label: PHONE_NUM, Score: 0.88


In [8]:
import re

EMAIL_REGEX = re.compile(r".+@.+\..+")
PHONE_DIGITS = re.compile(r"\d")

ENTITY_THRESHOLDS = {
    "NAME_STUDENT": 0.30,
    "EMAIL": 0.60,
    "PHONE_NUM": 0.60,
    "URL_PERSONAL": 0.70,
    "USERNAME": 0.50,
    "ID_NUM": 0.60,
    "STREET_ADDRESS": 0.40,
    "DATE": 0.60,
}

def validate_phone(text):
    digits = re.sub(r"\D", "", text)
    return 7 <= len(digits) <= 15

def postprocess_entities(text, entities):
    final = []
    for e in entities:
        label = e["entity_group"]
        score = e["score"]
        word = e["word"]

        if score < ENTITY_THRESHOLDS.get(label, 0.5):
            continue

        if label == "EMAIL" and not EMAIL_REGEX.match(word.replace(" ", "")):
            continue


        if label == "PHONE_NUM" and not validate_phone(word):
            continue

        final.append({
            "text": word,
            "type": label,
            "score": round(score, 2),
            "start": e["start"],
            "end": e["end"],
        })
    return final


In [41]:
import re

# create regular expressions for email and phone numbers
email_regex = re.compile(r".+@.+\..+")
phone_regex = re.compile(r'\d{3}-\d{3}-\d{4}$')

# I define entity_thresholds because when testing it define wrong pii types with low scores. Therefore using threholds help to prevent.
ENTITY_THRESHOLDS = {
    "NAME_STUDENT": 0.30,
    "EMAIL": 0.60,
    "PHONE_NUM": 0.60,
    "URL_PERSONAL": 0.70,
    "USERNAME": 0.50,
    "ID_NUM": 0.60,
    "STREET_ADDRESS": 0.40,
    "DATE": 0.60,
}

def validate_phone(text):
    digits = re.sub(r'\D', ',', text)
    return 7 <= len(digits) <= 15

def postprocess(text, entities):
  final = []
  for e in entities:
    label = e["entity_group"]
    score = e["score"]
    word = e["word"]
    if score < ENTITY_THRESHOLDS.get(label, 0.5):
      continue

    if label == "EMAIL" and not email_regex.match(word.replace(" ", "")):
          continue


    if label == "PHONE_NUM" and not validate_phone(word):
          continue

    final.append({
      "text": word,
      "type": label,
      "score": round(score, 2),
      "start": e["start"],
      "end": e["end"],
    })
  return final

In [42]:
def detect_pii(text):
    if not isinstance(text, str) or text.strip() == "":
        return []
    raw_entities = ner_pipeline(text)
    # return postprocess_entities(text, raw_entities)
    return postprocess(text, raw_entities)

df["pii_detected"] = df["user_input"].apply(detect_pii)

In [43]:
for i, row in df.head(40).iterrows():
    print("=" *10)
    print("\nTEXT:")
    print(row["user_input"])
    print("\nPII DETECTED:")
    if not row["pii_detected"]:
        print("  None")
    else:
        for e in row["pii_detected"]:
            print(f"  - {e['text']} → {e['type']} ({e['score']})")



TEXT:
Hi, I need help resetting my password. My email is sarah.johnson@email.com and my username is sjohnson_1985

PII DETECTED:
  - sarah. johnson @ email. com → EMAIL (0.8899999856948853)
  - sjohnson _ 1985 → USERNAME (0.7599999904632568)

TEXT:
What's the weather like today?

PII DETECTED:
  None

TEXT:
I'm trying to update my profile. My name is Michael Chen, phone number is 555-123-4567, and I live at 742 Maple Street, Springfield, IL 62701

PII DETECTED:
  - michael chen, → NAME_STUDENT (0.5799999833106995)
  - 742 maple street, springfield, il 62701 → STREET_ADDRESS (0.8100000023841858)

TEXT:
Can you explain how machine learning works?

PII DETECTED:
  None

TEXT:
Hello, I need to verify my account. My ID number is DL-8472-3901-5562 and my personal website is https://www.robertmiller-portfolio.com

PII DETECTED:
  - dl - 8472 - 3901 - 5562 → ID_NUM (0.9900000095367432)
  - https : / / www. robertmiller - portfolio. com → URL_PERSONAL (0.9200000166893005)

TEXT:
I can't access